In [1]:
%matplotlib inline
import re
import seaborn as sbn
import nltk
import tqdm as tqdm
import sqlite3
import pandas as pd
import numpy as np
nltk.download('stopwords')


import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

from sklearn.svm import LinearSVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sujaykumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('data/reviews_sports.csv')

/var/folders/l2/_mks1sd56yjdxq9514qt1fnh0000gn/T/ipykernel_7411/3608450076.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/reviews_sports.csv')


In [3]:
df = df.drop(['Unnamed: 0','style','vote','image'],axis=1)

<h2>As we are taking only verified Users we drop all the false Reviews</h2>

In [4]:
df["verified"].value_counts()

True     2611805
False     228135
Name: verified, dtype: int64

In [5]:
df = df[df['verified']]

In [6]:
df["verified"].value_counts()

True    2611805
Name: verified, dtype: int64

<h3>We Consider only the Text Based to Recommend the Products</h3>

In [7]:
df = df.drop(['reviewTime','reviewerName','unixReviewTime','verified'],axis=1)

In [8]:
df = df.dropna(axis=0)

In [9]:
df

,overall,reviewerID,asin,reviewText,summary
0,5.0,A180LQZBUWVOLF,0000032034,What a spectacular tutu! Very slimming.,Five Stars
1,1.0,ATMFGKU5SVEYY,0000032034,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!
2,5.0,A1QE70QBJ8U6ZG,0000032034,Exactly what we were looking for!,Five Stars
3,5.0,A22CP6Z73MZTYU,0000032034,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...
4,4.0,A22L28G8NRNLLN,0000032034,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...
...,...,...,...,...,...
2839935,5.0,A3TN0U64HONOPB,B01HJGAJ9O,"Love the shorts.. amazing comfortable, perfect...",Five Stars
2839936,5.0,AJDQLM8PT3YWT,B01HJGAJ9O,Superb quality,Five Stars
2839937,5.0,A3QK5ZLRE2KHLL,B01HJHHBHG,Works every time,Quality catcher
2839938,5.0,A3VDML80KNR9QQ,B01HJHHBHG,I have a briley bolt release paddle installed ...,Works for A400 with Briley Bolt Release


In [10]:
df['overall'].value_counts()

5.0    1777587
4.0     450064
3.0     192538
1.0      98020
2.0      91986
Name: overall, dtype: int64

<h3> Cleaning the Data (NLP)</h3>

In [11]:
def data_clean(rev, remove_stopwords=True):
    if isinstance(rev, str):
        new_text = re.sub("[^a-zA-Z]", " ", rev)
        words = new_text.lower().split()
        if remove_stopwords:
            sts = set(stopwords.words("english"))
            words = [w for w in words if not w in sts]
        ary = []
        eng_stemmer = english_stemmer
        for word in words:
            ary.append(eng_stemmer.stem(word))
        return ary
    else:
        return []

In [12]:
clean_reviewData = []
for rev in df['reviewText']:
    clean_reviewData.append( " ".join(data_clean(rev)))

clean_summaryData = []
for rev in df['summary']:
    clean_summaryData.append( " ".join(data_clean(rev)))

In [13]:
Most_used_Words_Review = pd.Series(' '.join(clean_reviewData).lower().split()).value_counts()[:20]
print(Most_used_Words_Review)

great      776312
use        769459
work       559887
good       537444
one        491060
well       462349
like       460035
fit        440421
get        376341
would      351072
product    307713
love       295839
nice       293101
look       270850
time       262942
price      259860
qualiti    253952
perfect    252113
need       249000
littl      236512
dtype: int64


In [14]:
Most_used_Words_Summary = pd.Series(' '.join(clean_summaryData).lower().split()).value_counts()[:20]
print (Most_used_Words_Summary)

star       722658
five       537931
great      374617
good       216510
work       116484
four       106393
nice       103430
product     88106
love        77590
qualiti     73176
price       70821
perfect     68781
fit         68724
use         60328
like        58735
well        57193
one         51356
best        45200
excel       42633
three       41810
dtype: int64


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_vectorizer = TfidfVectorizer(min_df=4, max_features = 1000)
test_vecor = text_vectorizer.fit_transform(clean_reviewData)
tfidf_vector = dict(zip(text_vectorizer.get_feature_names_out(), text_vectorizer.idf_))

In [16]:
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
sample_review = df.reviewText[:10]
for test in sample_review:
    test
    ss = analyser.polarity_scores(test)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]))
    print(test)

compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
What a spectacular tutu! Very slimming.
compound: -0.2287, 
neg: 0.05, 
neu: 0.95, 
pos: 0.0, 
What the heck? Is this a tutu for nuns? I know you can cut it but STILL. Also there aren't several layers of the tutu making it "poof out" It just lays flat. Needless to say it was returned.
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Exactly what we were looking for!
compound: 0.6983, 
neg: 0.018, 
neu: 0.901, 
pos: 0.081, 
I used this skirt for a Halloween costume and glued a bunch of feathers over it.  It was exactly what I needed.  I liked that the elastic waist didn't dig in (no muffin top).  Two things to note though:  (i) it is a little long; I cut about 8 inches off, but it was really easy to do; and (ii) it is see-thru b/c unlined - I had to get a pair of yellow shorts to wear underneath which I hadn't planned on doing.
compound: 0.3854, 
neg: 0.058, 
neu: 0.83, 
pos: 0.112, 
This is thick enough that you can't see through it, but

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sujaykumar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/var/folders/l2/_mks1sd56yjdxq9514qt1fnh0000gn/T/ipykernel_7411/3928246299.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sample_review = df.reviewText[:10]


In [17]:
test_reviewText = df.reviewText
test_Ratings = df.overall
text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(test_reviewText)
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [18]:
test_reviewText

0                    What a spectacular tutu! Very slimming.
1          What the heck? Is this a tutu for nuns? I know...
2                          Exactly what we were looking for!
3          I used this skirt for a Halloween costume and ...
4          This is thick enough that you can't see throug...
                                 ...                        
2839935    Love the shorts.. amazing comfortable, perfect...
2839936                                       Superb quality
2839937                                     Works every time
2839938    I have a briley bolt release paddle installed ...
2839939                                          Works great
Name: reviewText, Length: 2610195, dtype: object

In [25]:
test_Ratings

0          5.0
1          1.0
2          5.0
3          5.0
4          4.0
          ... 
2839935    5.0
2839936    5.0
2839937    5.0
2839938    5.0
2839939    5.0
Name: overall, Length: 2610195, dtype: float64

In [26]:
df = pd.DataFrame({
    'reviewText': test_reviewText,
    'Ratings': test_Ratings
})

In [ ]:
df.to_csv("processed_data/final_pre-processed.csv")

In [29]:
num_clients = list(range(1, 10 + 1))

In [32]:
df1 = pd.read_csv('data/reviews_sports.csv')

/var/folders/l2/_mks1sd56yjdxq9514qt1fnh0000gn/T/ipykernel_7411/3397604115.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data/reviews_sports.csv')


In [33]:
df1

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,vote,image
0,0,5.0,True,"06 3, 2015",A180LQZBUWVOLF,0000032034,Michelle A,What a spectacular tutu! Very slimming.,Five Stars,1433289600,NaN,NaN,NaN
1,1,1.0,True,"04 1, 2015",ATMFGKU5SVEYY,0000032034,Crystal R,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!,1427846400,NaN,NaN,NaN
2,2,5.0,True,"01 13, 2015",A1QE70QBJ8U6ZG,0000032034,darla Landreth,Exactly what we were looking for!,Five Stars,1421107200,NaN,NaN,NaN
3,3,5.0,True,"12 23, 2014",A22CP6Z73MZTYU,0000032034,L. Huynh,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...,1419292800,NaN,NaN,NaN
4,4,4.0,True,"12 15, 2014",A22L28G8NRNLLN,0000032034,McKenna,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...,1418601600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839935,2839935,5.0,True,"04 5, 2018",A3TN0U64HONOPB,B01HJGAJ9O,Lewis Perez,"Love the shorts.. amazing comfortable, perfect...",Five Stars,1522886400,"{'Size:': ' Small', 'Color:': ' Black/White'}",NaN,NaN
2839936,2839936,5.0,True,"03 1, 2018",AJDQLM8PT3YWT,B01HJGAJ9O,Andrew M. Silverman,Superb quality,Five Stars,1519862400,"{'Size:': ' Large', 'Color:': ' Black/Nero'}",NaN,NaN
2839937,2839937,5.0,True,"03 17, 2018",A3QK5ZLRE2KHLL,B01HJHHBHG,old hunter,Works every time,Quality catcher,1521244800,NaN,NaN,NaN
2839938,2839938,5.0,True,"10 28, 2017",A3VDML80KNR9QQ,B01HJHHBHG,PJT,I have a briley bolt release paddle installed ...,Works for A400 with Briley Bolt Release,1509148800,NaN,NaN,NaN
